In [ ]:
import mcstasscript as ms
my_configurator = ms.Configurator()
my_configurator.set_mcrun_path("/usr/bin/")
my_configurator.set_mcstas_path("/usr/share/mcstas/3.4/")

In [ ]:
instrument = ms.McStas_instr("my_instrument")
L0=2.165
DL=0.02
By=4.68e-3
AnaSign=1

In [ ]:
instrument.available_components()

In [ ]:
source = instrument.add_component("source", "Source_simple")
source.show_parameters()

In [ ]:
source.yheight = 0.01
source.xwidth = 0.01
source.dist = 5
source.focus_xw = 0.01
source.focus_yh = 0.01
source.lambda0 = 2.165
source.dlambda = 2.2
gauss=1
print(source)

In [ ]:
wavelength = instrument.add_parameter("wavelength", value=L0, comment="Wavelength in AA")
delta_wavelength = instrument.add_parameter("delta_wavelength", value=DL, comment="Stdev of Wavelength in AA")
By = instrument.add_parameter("By", value=By, comment="Flipper field strength in Y direction (?)")

source.lambda0 = wavelength
source.dlambda = delta_wavelength
print(source)

In [ ]:
set_pol = instrument.add_component("set_pol", "Set_pol")
set_pol.set_parameters(px=1)
set_pol.set_AT(0.01, RELATIVE=source)

In [ ]:
f1 = instrument.add_component("f1","Foil_flipper_magnet")
f1.set_parameters(
    xwidth=0.1, 
    yheight=0.1, 
    zdepth=0.3, 
    phi=0.095993, 
    foilthick=3.0, 
    Bx=0, 
    By=By, 
    Bz=0, 
    foil_in=1, verbose=0
)
f1.set_AT(1.0, RELATIVE=source)

In [ ]:
f2 = instrument.copy_component("f1")
f2.set_AT(3.0, RELATIVE=source)
# Todo: remove this, make it unnecessary
By=4.68e-3

f2.set_parameters(By=2 * By)


In [ ]:
ana = instrument.add_component("ana","PolAnalyser_ideal")
ana.set_parameters(mx=AnaSign)
ana.set_AT(0.18, RELATIVE=f2)


In [ ]:
sans_spheres = instrument.add_component("sans_sample", "SANS_spheres2")

In [ ]:
sans_spheres.set_AT(0.2, RELATIVE=f2)
sans_spheres.set_parameters(xwidth=0.02, 
    yheight=0.02, 
	zthick=0.001, 
	dsdw_inc=0.0, 
	sc_aim=0.8, 
	sans_aim=0.95, 
	R=10000, 
	phi=0.015, 
	singlesp=0,
	Qmind = 0.00003,
	Qmaxd = 0.001)
print(sans_spheres)


In [ ]:
PSD = instrument.add_component("PSD", "PSD_monitor")
PSD.set_AT([0, 0, 5], RELATIVE=source)
PSD.set_parameters(xwidth=0.011, yheight=0.011 ,ny=1001, nx=1, filename='"PSD.dat"')

In [ ]:
instrument.show_parameters()

In [ ]:
instrument.settings(mpi=4, ncount=1E8)

In [ ]:
data = instrument.backengine()

In [ ]:
ms.make_sub_plot(data, log=False)


In [ ]:
print(data)

In [ ]:
PSD_data = ms.name_search("PSD", data)

In [ ]:
PSD_data.Intensity